In [ ]:
import chess.pgn
import pandas as pd
import re
import ast

## Create dataframe (without moves for time efficiency)

In [ ]:
pgn = open("data/lichess_db_standard_rated_2014-07.pgn")
games = []

while True:
    game = chess.pgn.read_game(pgn)
    if game is None:
        break
    games.append(game)

In [ ]:
games_data = []

for game in games:
    headers = game.headers
    games_data.append({
        "White": headers["White"],
        "Black": headers["Black"],
        "Result": headers["Result"],
        "Opening": headers["Opening"],
        "WhiteElo": headers["WhiteElo"],
        "BlackElo": headers["BlackElo"],
        "TimeControl": headers["TimeControl"],
        "Termination": headers["Termination"],
        
    })

df = pd.concat([pd.DataFrame(games_data)], ignore_index=True)

In [46]:
df["Result"] = df["Result"].replace("1-0", "White")
df["Result"] = df["Result"].replace("0-1", "Black")
df["Result"] = df["Result"].replace("1/2-1/2", "Draw")
df = df.rename(columns={"Result": "Winner"})

In [ ]:
df.to_csv("data/games/games_without_moves.csv", index=False)

## Update dataset to include moves

In [3]:
with open("data/lichess_db_standard_rated_2014-07.pgn") as pgn:
    lines = pgn.readlines()
    moves_lines = [line for line in lines if line.startswith("1.") or line.startswith(" 0") or line.startswith(" 1")]

In [5]:
games = pd.DataFrame(moves_lines, columns=['Moves'])

In [ ]:
games['Moves'] = games['Moves'].apply(lambda x: re.sub(r'\d+\.', '', x))
games['Moves'] = games['Moves'].apply(lambda x: re.sub(r'\{ \[%eval [^\}]*\}', '', x))
games['Moves'] = games['Moves'].apply(lambda x: re.sub(r'\.\.', '', x))
games['Moves'] = games['Moves'].apply(lambda x: re.sub(r'[!?]', '', x))
games['Moves'] = games['Moves'].apply(lambda x: re.sub(r'\s+', ' ', x))
games['Moves'] = games['Moves'].apply(lambda x: x.strip())
games['Moves'] = games['Moves'].apply(lambda x: x.split(' ')[:-1])

In [ ]:
no_games = pd.read_csv("data/games/games_without_moves.csv")

/var/folders/jp/06cx0jr53fn1xtpgl_6nbcpc0000gn/T/ipykernel_86470/4203723164.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  no_games = pd.read_csv("data/games/games_july_2014.csv")


In [ ]:
games = pd.concat([no_games, games], axis=1)

In [24]:
games['Moves'] = games['Moves'].apply(lambda x: ast.literal_eval(x))

In [ ]:
games['Moves'] = games['Moves'].apply(lambda x: x if len(x) >= 10 else None)
games = games.dropna()
games['Moves'] = games['Moves'].apply(lambda x: x[:10])

In [29]:
games.to_csv("data/games/cleaned_games.csv", index=False)